 Importing Librares

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
import keras
import os
from tqdm import tqdm
from keras.preprocessing.image import img_to_array
import pandas as pd
from tensorflow.keras.layers import BatchNormalization
import random
import cv2  
import imutils
from skimage import color
from skimage import io


# Data Preprocessing

## X_Data

In [ ]:
SIZE= 250
Path1='/content/drive/MyDrive/διπλωματικη/data/my data/images'
Path2='/content/drive/MyDrive/διπλωματικη/data/my data/moires/file.csv'
files=os.listdir(Path1)
train_y = []
eikones=[]


In [ ]:
for i in tqdm(range(1,1900)):
  i=str(i)
  image=cv2.imread(Path1+'/'+'image-'+i+'.png',1)
  image = imutils.rotate(image, angle= 180)
  #img=cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  img=cv2.resize(image,(SIZE, SIZE))
  train_y.append(img_to_array(img))
  eikones.append(image)

In [ ]:
print(train_y[0].shape)

In [ ]:
image_array = np.reshape(train_y, (len(train_y), SIZE, SIZE, 3))
image_array = image_array.astype('float32') / 255.
train_array= image_array[:1850,:,:]
test_array=image_array[1851:,:,:]
#val_array=image_array[3301:,:,:]

In [ ]:
print(image_array.shape)

### Show my images

In [ ]:
figure =plt.figure(dpi=1200)
plt.subplot(231)
plt.title('Sample 1')
plt.imshow(eikones[0], cmap='gray',origin="lower")
plt.axis("off")
plt.subplot(232)
plt.title('Sample 2')
plt.imshow(eikones[1], cmap='gray',origin="lower")
plt.axis("off")
plt.subplot(233)
plt.title('Sample 3')
plt.imshow(eikones[1498], cmap='gray',origin="lower")
plt.axis("off")
plt.subplot(234)
plt.title('Sample 4')
plt.imshow(eikones[1498], cmap='gray',origin="lower")
plt.axis("off")
plt.show()

## Y_Data

In [ ]:
dataset = pd.read_csv(Path2)
y = dataset.iloc[:, -1].values
y=-y


In [ ]:
for i in range(0,1499):
  print(y[i])

In [ ]:
train_y_array= y[:1850]
test_y_array=y[1851:1900]
#val_y_array=y[3301:]



# Model

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import numpy
from keras.layers import Input
from keras.layers import UpSampling2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose 
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger,ReduceLROnPlateau
import pickle
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.applications import VGG16

In [ ]:
SIZE=250
cnn = Sequential()
inputs =Input(shape=(SIZE,SIZE,3))
base_model = tf.keras.applications.MobileNet (weights = 'imagenet', include_top = False
                )
base_model.trainable = False
cnn = base_model(inputs,training=False)
base_model.summary()

In [ ]:
from keras.layers.core import activation
from tensorflow.keras.layers import BatchNormalization
SIZE=250
c= tf.keras.layers.GlobalMaxPooling2D()(cnn)
c=tf.keras.layers.Flatten()(c)
c=tf.keras.layers.Dense(512, activation='relu')(c)
c=tf.keras.layers.Dropout(0.1)(c)
c=tf.keras.layers.Dense(256, activation='relu')(c)
c=tf.keras.layers.Dropout(0.2)(c)
c=tf.keras.layers.Dense(64, activation='relu')(c)
c=tf.keras.layers.Dropout(0.2)(c)
output=tf.keras.layers.Dense(1, activation=tf.keras.activations.linear)(c)
myfriend= Model(inputs=inputs,outputs=output)
myfriend.summary()



# Fit My Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)
myfriend.compile(optimizer = optimizer,loss = 'mse')
callbacks= [keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/διπλωματικη/models/ResNet.hdf5',verbose = 1,monitor="loss",
save_best_only = True),EarlyStopping(monitor='loss',patience=10,verbose=1,restore_best_weights=True),
CSVLogger('/content/drive/MyDrive/διπλωματικη/models/Logmyfriend.csv'),
ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=10,verbose=1,min_lr=0.0002)]
results=myfriend.fit(train_array,train_y_array,batch_size=24,epochs=100,callbacks=callbacks,validation_split=0.1)

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/διπλωματικη/models/ResNet.hdf5')

# Feature visualization 1

In [ ]:
layer_names = [layer.name for layer in myfriend.layers]
layer_names

In [ ]:
layer_outputs = [layer.output for layer in myfriend.layers]
layer_outputs

In [ ]:
feature_map_model = tf.keras.models.Model(inputs=myfriend.input, outputs=layer_outputs)

In [ ]:
image_path='/content/drive/MyDrive/διπλωματικη/data/chech.png'
image=cv2.imread(image_path,0)
image=cv2.resize(image,(250, 250))
input=img_to_array(image)
input = np.reshape(input, (1, SIZE, SIZE, 1))
input = input.astype('float32') / 255.0


In [ ]:
feature_maps = feature_map_model.predict(input)

In [ ]:
for layer_name, feature_map in zip(layer_names, feature_maps):
  print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")

In [ ]:
for layer_name, feature_map in zip(layer_names, feature_maps):  
  if len(feature_map.shape) == 4:
      k = feature_map.shape[-1]  
      size=feature_map.shape[1]
      image_belt = np.zeros((size, size*k),np.uint8)
      for i in range(k):
        feature_image = feature_map[0, :, :, i]
        feature_image-= feature_image.mean()
        feature_image/= feature_image.std ()
        feature_image*=  64
        feature_image+= 128
        feature_image= np.clip(feature_image, 0, 255).astype('uint8')
        image_belt[:, i * size : (i + 1) * size] = feature_image
        scale = 20. / k
        plt.figure( figsize=(scale * k, scale) )
        plt.title ( layer_name )
        plt.grid  ( False )
        plt.imshow( image_belt,cmap='gray')
      #columns = size
      #rows = size
      #for ftr in feature_map:
        #fig=plt.figure(figsize=(12, 12))
        #for i in range(1,k):
          #fig =plt.subplot(rows, columns, i)
          #fig.set_xticks([])  #Turn off axis
          #fig.set_yticks([])
          #plt.imshow(ftr[ :, :, i-1], cmap='gray')
          
        #plt.show()

# Prediction

In [ ]:
test_img_input=np.expand_dims(train_array[0], 0)
pred = model.predict(test_img_input)
#angle=sc.inverse_transform(pred)[0][0]
#print(angle)
print('original degrees:',train_y_array[0])
#angle=round(pred,0)
print('predicted degrees:',pred)
  
 

## Average Error Deviation

### Known Data

In [ ]:
known_data=[]

In [ ]:
CE=0

In [ ]:
from sklearn.preprocessing import StandardScaler
import math
MAE=0
#scaler = StandardScaler()
for i in tqdm(range(0,1850)):
  test_img_input=np.expand_dims(train_array[i], 0)
  pred = model.predict(test_img_input)
#angle=sca.inverse_transform(pred)[0][0]
  #print(pred)
#angle=round(pred,0)
#print(prend)
  
  U=train_y_array[i]-pred
  tet=abs(U)
  known_data.append(tet)
  if tet<1:
    CE=CE+1
  MAE=MAE+tet
print(MAE)

In [ ]:
print(MAE/1850)

[[2.9341595]]


### Unknown Data

In [ ]:
unknown_data=[]

In [ ]:
CE_un=0

In [ ]:
from sklearn.preprocessing import StandardScaler
import math
MSE_t=0
#scaler = StandardScaler()
for i in tqdm(range(0,48)):
  test_img_input=np.expand_dims(test_array[i], 0)
  pred = model.predict(test_img_input)
#angle=sca.inverse_transform(pred)[0][0]
  #print(pred)
#angle=round(pred,0)
#print(prend)
  
  U_t=test_y_array[i]-pred
  tet=abs(U_t)
  unknown_data.append(tet)
  if tet<1:
    CE_un=CE_un+1
  MSE_t=MSE_t+tet
print(MSE_t)

In [ ]:
print(MSE_t/48)

## Top80

### Known Data

In [ ]:
print(known_data)

In [ ]:
known_data.sort()

In [ ]:
Top80_list=[]

In [ ]:
for i in range(0,1480):
  Top80_list.append(known_data[i])

In [ ]:
Top80=0

In [ ]:
for i in range(0,1480):
  Top80=Top80 + Top80_list[i]

In [ ]:
Top80=Top80 /1480

In [ ]:
print(Top80)

### Unknown Data

In [ ]:
unknown_data.sort()

In [ ]:
Top80_Unknown_list=[]

In [ ]:
for i in range(0,39):
  Top80_Unknown_list.append(unknown_data[i])

In [ ]:
Top80_unknown=0

In [ ]:
for i in range(0,39):
  Top80_unknown=Top80_unknown + Top80_Unknown_list[i]

In [ ]:
Top80_unknown=Top80_unknown/39

In [ ]:
print(Top80_unknown)

[[2.138294]]


## CE_criterion

### Known Data

In [ ]:
print(CE)

In [ ]:
CE_criterion=CE/1850

In [ ]:
print(CE_criterion)

### unknown data

In [ ]:
CE_un_criterion=CE_un/48

In [ ]:
print(CE_un_criterion)

# Plot loss

In [ ]:
loss = results.history['loss']
val_loss = results.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()